# Pipeline Completo: Análisis de CT Pulmonar

**Notebook 06** - Pipeline integrado que combina todos los módulos del proyecto

Este notebook ejecuta el flujo completo de análisis de imágenes CT pulmonares, integrando los módulos desarrollados en los notebooks anteriores:

---

## Estructura del Pipeline

| Módulo | Notebook | Descripción |
|--------|----------|-------------|
| 1. Preparación de datos | `00_preparacion_datos.ipynb` | Descarga LUNA16, verificación LIDC |
| 2. Preprocesamiento | `01_preprocesamiento.ipynb` | Máscaras de pulmón y nódulos LIDC |
| 3. Visualización | `02_visualizacion.ipynb` | Visualización 2D/3D |
| 4. Segmentación | `03_nodulos.ipynb` | U-Net para nódulos |
| 5. Clasificación | `04_clasificacion.ipynb` | Benigno/Maligno |
| 6. Denoising | `05_denoising.ipynb` | Simulación ruido LDCT, DnCNN |

---

## Configuración del Entorno

In [ ]:
# Detectar entorno de ejecucion
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("[INFO] Ejecutando en Google Colab")
    print("="*50)
    
    # Instalar dependencias
    print("\n[INFO] Instalando dependencias...")
    import subprocess
    paquetes = ['SimpleITK', 'scikit-image', 'requests', 'tqdm']
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + paquetes)
    
    # Clonar repositorio desde GitHub
    print("\n[INFO] Clonando repositorio desde GitHub...")
    repo_url = "https://github.com/Daspony/Imagenes-Biomedicas.git"
    repo_name = "Imagenes-Biomedicas"
    
    if not os.path.exists(f"/content/{repo_name}"):
        subprocess.run(["git", "clone", repo_url], cwd="/content", check=True)
        print(f"[OK] Repositorio clonado en /content/{repo_name}")
    else:
        print(f"[OK] Repositorio ya existe en /content/{repo_name}")
    
    # Anadir al path
    sys.path.insert(0, f"/content/{repo_name}")
    
    print("[OK] Configuracion de Colab completada\n")
    
else:
    print("[INFO] Ejecutando localmente")
    print("="*50)
    
    # Anadir directorio padre al path para importar utils
    parent_dir = os.path.abspath('..')
    if parent_dir not in sys.path:
        sys.path.insert(0, parent_dir)
    
    print(f"[INFO] Directorio de trabajo: {os.getcwd()}")
    print("[OK] Configuracion local completada\n")

## Importar Módulos del Proyecto

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings

# Importar módulos del proyecto
from utils import LUNA16DataLoader, LungPreprocessor, LungVisualizer, SegmentationMetrics
from utils import download_luna16
from utils.lidc_loader import LIDCAnnotationLoader

# Compatibilidad numpy para pylidc
np.int = np.int64
np.float = np.float64

# Configurar matplotlib
plt.rcParams['figure.dpi'] = 100

print("Módulos disponibles:")
print("  - LUNA16DataLoader: Carga de datos .mhd/.raw")
print("  - LungPreprocessor: Segmentación pulmonar")
print("  - LIDCAnnotationLoader: Anotaciones LIDC-IDRI")
print("  - LungVisualizer: Visualización")
print("  - SegmentationMetrics: Métricas de evaluación")

In [ ]:
# Importar modulo de descarga
from pathlib import Path
from utils import download_luna16

# Verificar si LUNA16 ya existe
project_root = os.path.abspath('..')  # Directorio padre de notebooks/
luna16_path = os.path.join(project_root, 'LUNA16')
subset0_path = os.path.join(luna16_path, 'subset0')
annotations_path = os.path.join(luna16_path, 'annotations.csv')

print("="*70)
print("VERIFICACION DE DATOS LUNA16")
print("="*70)
print(f"\n[INFO] Buscando datos en: {luna16_path}")

# Verificar si subset0 existe y tiene archivos .mhd
data_exists = False
if os.path.exists(subset0_path):
    mhd_files = list(Path(subset0_path).glob("*.mhd"))
    if len(mhd_files) > 0:
        data_exists = True
        print(f"\n[OK] Datos encontrados: {len(mhd_files)} archivos .mhd en subset0")
        
        # Verificar anotaciones
        if os.path.exists(annotations_path):
            print(f"[OK] Archivo de anotaciones encontrado")
        else:
            print(f"[WARNING] Archivo de anotaciones no encontrado (se descargara)")
    else:
        print(f"\n[WARNING] Carpeta subset0 existe pero esta vacia")
else:
    print(f"\n[WARNING] No se encontro la carpeta subset0")

# Si no existen los datos, ofrecer descarga
if not data_exists:
    print("\n" + "="*70)
    print("DESCARGA AUTOMATICA DE DATOS")
    print("="*70)
    print("\n[INFO] No se encontraron datos LUNA16 en el sistema.")
    print("\n[INFO] Opcion de descarga automatica:")
    print("  - subset0: ~13GB (89 escaneos CT)")
    print("  - annotations.csv: Anotaciones de nodulos")
    print("  - candidates.csv: Candidatos a nodulos")
    print(f"\n[INFO] Destino: {luna16_path}")
    
    if IN_COLAB:
        print("\n[WARNING] ATENCION: La descarga consumira ~13GB de espacio en Google Drive")
        user_input = input("\nDeseas descargar subset0 ahora? (s/n): ").strip().lower()
        should_download = user_input in ['s', 'si', 'yes', 'y']
    else:
        print("\n[WARNING] ATENCION: La descarga consumira ~13GB de espacio en disco")
        print("Tiempo estimado: 30-60 minutos dependiendo de tu conexion")
        user_input = input("\nDeseas descargar subset0 ahora? (s/n): ").strip().lower()
        should_download = user_input in ['s', 'si', 'yes', 'y']
    
    if should_download:
        print("\n[INFO] Iniciando descarga...")
        print("Puedes cancelar con Ctrl+C en cualquier momento\n")
        
        success = download_luna16(
            subsets=0,
            include_csv=True,
            download_dir=luna16_path
        )
        
        if success:
            print("\n" + "="*70)
            print("[OK] DESCARGA COMPLETADA EXITOSAMENTE")
            print("="*70)
        else:
            print("\n" + "="*70)
            print("[ERROR] ERROR EN LA DESCARGA")
            print("="*70)
            raise RuntimeError("Descarga de LUNA16 fallo.")
    else:
        print("\n[INFO] Descarga omitida.")
        raise RuntimeError("Datos LUNA16 no disponibles.")
else:
    print("\n[OK] Datos LUNA16 verificados correctamente\n")

---

# MÓDULO 1: Preparación de Datos

> Ver detalles en: `00_preparacion_datos.ipynb`

Descarga automática de LUNA16 y verificación de anotaciones LIDC.

## 1.1 Configurar rutas y descargar datos

In [ ]:
from pathlib import Path

# Configurar rutas
if IN_COLAB:
    luna16_path = '/content/LUNA16'
else:
    project_root = os.path.abspath('..')
    luna16_path = os.path.join(project_root, 'LUNA16')

# Descargar datos si no existen
download_luna16(subsets=0, include_csv=True, download_dir=luna16_path)

# Configurar paths
DATA_PATH = os.path.join(luna16_path, 'subset0')
ANNOTATIONS_PATH = os.path.join(luna16_path, 'annotations.csv')

# Obtener primer archivo disponible
mhd_files = list(Path(DATA_PATH).glob("*.mhd"))
EXAMPLE_SCAN = mhd_files[0].name if mhd_files else None

print(f"Datos configurados:")
print(f"  - Path: {DATA_PATH}")
print(f"  - Archivos .mhd: {len(mhd_files)}")
print(f"  - Ejemplo: {EXAMPLE_SCAN[:50]}..." if EXAMPLE_SCAN else "  - No hay archivos")

---

# MÓDULO 2: Preprocesamiento

> Ver detalles en: `01_preprocesamiento.ipynb`

Incluye:
- Carga de imágenes CT
- Segmentación pulmonar
- Máscaras de nódulos LIDC-IDRI

## 2.1 Cargar escaneo CT

In [ ]:
# Crear instancia del cargador
loader = LUNA16DataLoader(DATA_PATH, ANNOTATIONS_PATH)

print("[OK] LUNA16DataLoader inicializado")

## 2.2 Cargar escaneo CT

In [ ]:
# Ruta completa al archivo
scan_path = os.path.join(DATA_PATH, EXAMPLE_SCAN)

# Cargar imagen CT
print("[INFO] Cargando escaneo CT...")
ct_scan, origin, spacing = loader.load_itk_image(scan_path)

print("\n[OK] Escaneo cargado correctamente")
print("\nInformacion del volumen:")
print(f"  - Shape: {ct_scan.shape} (slices, height, width)")
print(f"  - Tipo de datos: {ct_scan.dtype}")
print(f"  - Tamano en memoria: {ct_scan.nbytes / (1024**2):.1f} MB")
print(f"\nInformacion espacial:")
print(f"  - Spacing: {spacing} mm (z, y, x)")
print(f"  - Origin: {origin} mm")
print(f"  - Dimensiones fisicas: {ct_scan.shape * spacing} mm")
print(f"\nRango de valores (Hounsfield Units):")
print(f"  - Minimo: {ct_scan.min():.1f} HU")
print(f"  - Maximo: {ct_scan.max():.1f} HU")
print(f"  - Media: {ct_scan.mean():.1f} HU")
print(f"  - Desviacion estandar: {ct_scan.std():.1f} HU")

## 2.3 Cargar anotaciones LIDC-IDRI

In [ ]:
# Extraer seriesuid del nombre del archivo
seriesuid = EXAMPLE_SCAN.replace('.mhd', '')

# Inicializar cargador LIDC
lidc_loader = LIDCAnnotationLoader()

# Obtener nódulos confiables (≥3 radiólogos)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    reliable_nodules = lidc_loader.get_reliable_nodules(seriesuid, min_annotations=3)

print(f"Nódulos confiables encontrados: {len(reliable_nodules)}")
for i, nodule in enumerate(reliable_nodules):
    malignancy = lidc_loader.get_consensus_malignancy(nodule)
    print(f"  Nódulo {i+1}: {len(nodule)} anotaciones, malignidad = {malignancy:.1f}/5")

## 2.4 Segmentación pulmonar y máscaras de nódulos

In [ ]:
# Crear instancia del preprocesador
preprocessor = LungPreprocessor()

# Seleccionar slice del medio
slice_idx = ct_scan.shape[0] // 2
ct_slice = ct_scan[slice_idx]

# Segmentar pulmones
lung_mask = preprocessor.segment_lung_mask(ct_slice, threshold=-320)

# Extraer máscaras de nódulos LIDC alineadas
nodule_masks = np.zeros(ct_scan.shape, dtype=np.uint8)
nodule_info = []

if reliable_nodules:
    print("Extrayendo máscaras de nódulos...")
    for idx in range(len(reliable_nodules)):
        result = lidc_loader.get_aligned_consensus_mask(
            seriesuid=seriesuid,
            nodule_idx=idx,
            origin=origin,
            spacing=spacing,
            ct_shape=ct_scan.shape,
            threshold=0.5
        )
        if result:
            mask, bbox = result
            nodule_masks = np.maximum(nodule_masks, mask)
            nodule_info.append({'idx': idx, 'bbox': bbox, 'volume': int(np.sum(mask))})
            print(f"  Nódulo {idx+1}: {np.sum(mask)} voxels")

print(f"\nMáscaras extraídas: {len(nodule_info)} nódulos")

## 2.2 Seleccionar Slice de Trabajo

In [ ]:
# Seleccionar slice del medio del volumen
slice_idx = ct_scan.shape[0] // 2
ct_slice = ct_scan[slice_idx]

print(f"[INFO] Slice seleccionado: {slice_idx}/{ct_scan.shape[0]}")
print(f"  - Shape: {ct_slice.shape}")
print(f"  - Rango HU: [{ct_slice.min():.1f}, {ct_slice.max():.1f}]")

## 2.3 Segmentación Pulmonar

In [ ]:
# Segmentar pulmones
print("[INFO] Segmentando region pulmonar...")
lung_mask = preprocessor.segment_lung_mask(ct_slice, threshold=-320)

# Calcular área pulmonar
area_pixels = np.sum(lung_mask)
area_mm2 = area_pixels * spacing[1] * spacing[2]
area_cm2 = area_mm2 / 100

print("\n[OK] Segmentacion completada")
print(f"  - Pixeles pulmonares: {area_pixels}")
print(f"  - Area pulmonar: {area_cm2:.1f} cm2")
print(f"  - Porcentaje del slice: {100 * area_pixels / ct_slice.size:.1f}%")

## 2.4 Normalización de Valores HU

In [ ]:
# Normalizar a rango [0, 1]
print("[INFO] Normalizando valores Hounsfield...")
ct_normalized = loader.normalize_hu(ct_slice, min_hu=-1000, max_hu=400)

print("\n[OK] Normalizacion completada")
print(f"  - Rango original: [{ct_slice.min():.1f}, {ct_slice.max():.1f}] HU")
print(f"  - Rango normalizado: [{ct_normalized.min():.3f}, {ct_normalized.max():.3f}]")

## 2.5 CLAHE (Opcional - para visualización mejorada)

In [ ]:
# Aplicar CLAHE para realzar contraste
print("[INFO] Aplicando CLAHE...")
ct_clahe = preprocessor.apply_clahe(ct_normalized, clip_limit=2.0, tile_size=(8, 8))

print("[OK] CLAHE aplicado")
print("  (Util para visualizar nodulos de baja densidad)")

---

# MÓDULO 3: Visualización

> Ver detalles en: `02_visualizacion.ipynb`

## 3.1 Pipeline de preprocesamiento

In [ ]:
# Normalizar CT
ct_normalized = loader.normalize_hu(ct_slice, min_hu=-1000, max_hu=400)

# Visualizar pipeline completo
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# 1. CT Original
axes[0].imshow(ct_slice, cmap='bone')
axes[0].set_title('1. CT Original (HU)')
axes[0].axis('off')

# 2. Normalizado
axes[1].imshow(ct_normalized, cmap='bone')
axes[1].set_title('2. Normalizado [0,1]')
axes[1].axis('off')

# 3. Máscara pulmonar
axes[2].imshow(ct_slice, cmap='bone')
axes[2].imshow(lung_mask, cmap='Greens', alpha=0.3)
axes[2].set_title('3. Máscara Pulmonar')
axes[2].axis('off')

# 4. Región pulmonar
ct_lung_only = ct_normalized * lung_mask
axes[3].imshow(ct_lung_only, cmap='bone')
axes[3].set_title('4. Región Pulmonar')
axes[3].axis('off')

# 5. Máscara de nódulos (si hay)
axes[4].imshow(ct_slice, cmap='bone')
if np.any(nodule_masks[slice_idx]):
    axes[4].imshow(nodule_masks[slice_idx], cmap='Reds', alpha=0.6)
axes[4].set_title('5. Nódulos LIDC')
axes[4].axis('off')

# 6. Combinado
axes[5].imshow(ct_slice, cmap='bone')
axes[5].imshow(lung_mask, cmap='Greens', alpha=0.2)
if np.any(nodule_masks[slice_idx]):
    axes[5].imshow(nodule_masks[slice_idx], cmap='Reds', alpha=0.6)
axes[5].set_title('6. Pulmón + Nódulos')
axes[5].axis('off')

plt.suptitle(f'Pipeline de Preprocesamiento - Slice {slice_idx}', fontsize=14)
plt.tight_layout()
plt.show()

## 3.2 Visualización de Nódulos Anotados

In [ ]:
if len(annotations_voxel) > 0:
    print(f"[INFO] Visualizando {len(annotations_voxel)} nodulos anotados...\n")
    
    # Seleccionar el primer nódulo
    nodule = annotations_voxel[0]
    nodule_slice_idx = nodule['z']
    
    # Cargar slice que contiene el nódulo
    ct_slice_nodule = ct_scan[nodule_slice_idx]
    lung_mask_nodule = preprocessor.segment_lung_mask(ct_slice_nodule)
    
    # Preparar anotaciones para visualización
    slice_annotations = [{
        'x': nodule['x'],
        'y': nodule['y'],
        'diameter': nodule['diameter'] / spacing[1]  # Convertir mm a píxeles
    }]
    
    # Visualizar usando el módulo visualizer
    visualizer.plot_ct_with_annotations(
        ct_slice_nodule,
        lung_mask=lung_mask_nodule,
        annotations=slice_annotations,
        title=f"Nodulo de {nodule['diameter']:.1f}mm - Slice {nodule_slice_idx}",
        figsize=(18, 6)
    )
    
    print(f"[OK] Nodulo visualizado en slice {nodule_slice_idx}")
    print(f"  - Coordenadas (voxel): Z={nodule['z']}, Y={nodule['y']}, X={nodule['x']}")
    print(f"  - Diametro: {nodule['diameter']:.2f} mm")
    
else:
    print("[INFO] No hay nodulos anotados en este escaneo para visualizar")

## 3.3 Visualización de Múltiples Slices del Volumen

In [ ]:
# Visualizar 9 slices distribuidos uniformemente
print("[INFO] Visualizando multiples slices del volumen...\n")

visualizer.plot_volume_slices(
    ct_scan,
    num_slices=9,
    cmap='bone',
    title=f'Volumen CT Completo - {seriesuid[:30]}...'
)

print("[OK] Visualizacion de volumen completada")

---

# MÓDULO 4: Métricas y Análisis

> Ver detalles en: `04_clasificacion.ipynb` y `05_denoising.ipynb`

## 4.1 Análisis de distribución HU

In [ ]:
# Analizar distribución de valores HU en región pulmonar
lung_hu_values = ct_slice[lung_mask == 1]

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histograma completo del slice
axes[0].hist(ct_slice.flatten(), bins=100, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(-1000, color='red', linestyle='--', linewidth=2, label='Aire (-1000 HU)')
axes[0].axvline(-320, color='orange', linestyle='--', linewidth=2, label='Threshold (-320 HU)')
axes[0].axvline(0, color='blue', linestyle='--', linewidth=2, label='Agua (0 HU)')
axes[0].set_xlabel('Unidades Hounsfield (HU)', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)
axes[0].set_title('Distribucion HU - Slice Completo', fontsize=14, weight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Histograma solo región pulmonar
axes[1].hist(lung_hu_values, bins=50, color='lightcoral', alpha=0.7, edgecolor='black')
axes[1].axvline(lung_hu_values.mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Media: {lung_hu_values.mean():.1f} HU')
axes[1].set_xlabel('Unidades Hounsfield (HU)', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)
axes[1].set_title('Distribucion HU - Solo Region Pulmonar', fontsize=14, weight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("[INFO] Estadisticas de la region pulmonar:")
print(f"  - Media: {lung_hu_values.mean():.1f} HU")
print(f"  - Desviacion estandar: {lung_hu_values.std():.1f} HU")
print(f"  - Mediana: {np.median(lung_hu_values):.1f} HU")
print(f"  - Percentil 25: {np.percentile(lung_hu_values, 25):.1f} HU")
print(f"  - Percentil 75: {np.percentile(lung_hu_values, 75):.1f} HU")

## 4.2 Evolución del área pulmonar

In [ ]:
# Calcular área pulmonar en cada slice (muestreado)
print("[INFO] Calculando evolucion del area pulmonar...\n")

lung_areas = []
slice_positions = []

for i in range(0, ct_scan.shape[0], 5):  # Cada 5 slices para velocidad
    lung_mask_temp = preprocessor.segment_lung_mask(ct_scan[i])
    area_mm2 = np.sum(lung_mask_temp) * spacing[1] * spacing[2]
    lung_areas.append(area_mm2 / 100)  # Convertir a cm2
    slice_positions.append(i)

# Graficar evolución
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(slice_positions, lung_areas, linewidth=2, color='steelblue', marker='o', markersize=4)
ax.fill_between(slice_positions, lung_areas, alpha=0.3, color='steelblue')

# Marcar máximo
max_area_idx = np.argmax(lung_areas)
max_slice = slice_positions[max_area_idx]
max_area = lung_areas[max_area_idx]
ax.axvline(max_slice, color='red', linestyle='--', alpha=0.7, linewidth=2)
ax.plot(max_slice, max_area, 'r*', markersize=20, label=f'Max area: {max_area:.0f} cm2 (slice {max_slice})')

ax.set_xlabel('Numero de Slice', fontsize=12)
ax.set_ylabel('Area Pulmonar (cm2)', fontsize=12)
ax.set_title('Evolucion del Area Pulmonar a lo Largo del Volumen CT', fontsize=14, weight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

plt.tight_layout()
plt.show()

print("[OK] Analisis de area pulmonar completado")
print(f"\n[INFO] Resumen:")
print(f"  - Area maxima: {max_area:.1f} cm2 (slice {max_slice})")
print(f"  - Area media: {np.mean(lung_areas):.1f} cm2")
print(f"  - Volumen pulmonar estimado: {np.sum(lung_areas) * spacing[0] / 10:.1f} cm3")

---

# Resumen del Pipeline

In [ ]:
print("="*60)
print("RESUMEN DEL PIPELINE COMPLETO")
print("="*60)

print("\nMÓDULOS EJECUTADOS:")
print("  1. Preparación de datos")
print(f"     - Archivos .mhd disponibles: {len(mhd_files)}")
print("  2. Preprocesamiento")
print(f"     - Volumen CT: {ct_scan.shape}")
print(f"     - Nódulos LIDC: {len(nodule_info)}")
print("  3. Visualización")
print("     - Pipeline de preprocesamiento visualizado")
print("  4. Métricas")
print("     - Distribución HU analizada")

print("\nDATOS PROCESADOS:")
print(f"  - SeriesUID: {seriesuid[:40]}...")
print(f"  - Spacing: {spacing} mm")
print(f"  - Nódulos confiables: {len(reliable_nodules)}")
print(f"  - Volumen total nódulos: {np.sum(nodule_masks)} voxels")

print("\nNOTEBOOKS RELACIONADOS:")
print("  - 00_preparacion_datos.ipynb: Descarga y verificación")
print("  - 01_preprocesamiento.ipynb: Detalles de preprocesamiento")
print("  - 02_visualizacion.ipynb: Visualización avanzada")
print("  - 03_nodulos.ipynb: Segmentación U-Net")
print("  - 04_clasificacion.ipynb: Clasificación benigno/maligno")
print("  - 05_denoising.ipynb: Simulación ruido y denoising")

print("\n" + "="*60)
print("PIPELINE COMPLETADO")
print("="*60)

## Próximos Pasos

### Para profundizar en cada módulo:

| Notebook | Contenido |
|----------|-----------|
| `01_preprocesamiento.ipynb` | Experimentación con thresholds, clear_border(), CLAHE |
| `02_visualizacion.ipynb` | Visualización 3D, controles interactivos |
| `03_nodulos.ipynb` | Arquitectura U-Net para segmentación |
| `04_clasificacion.ipynb` | ResNet18 para clasificación benigno/maligno |
| `05_denoising.ipynb` | Simulación LDCT, DnCNN, métricas PSNR/SSIM |

### Para usar en scripts propios:

```python
from utils import LUNA16DataLoader, LungPreprocessor
from utils.lidc_loader import LIDCAnnotationLoader

# Cargar datos
loader = LUNA16DataLoader(data_path, annotations_path)
ct_scan, origin, spacing = loader.load_itk_image(scan_path)

# Obtener máscaras de nódulos LIDC
lidc = LIDCAnnotationLoader()
nodules = lidc.get_reliable_nodules(seriesuid, min_annotations=3)
mask = lidc.get_aligned_consensus_mask(seriesuid, 0, origin, spacing, ct_scan.shape)
```